In [2]:
import pandas as pd

# Load dataset
df = pd.read_csv("raw_jobs_data.csv")

# Preview data
df.head()


,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


Filter Canada-only jobs

In [3]:
df_canada = df[df["company_location"] == "CA"]


Select relevant columns

In [4]:
df_canada = df_canada[
    [
        "job_title",
        "experience_level",
        "employment_type",
        "salary_in_usd",
        "work_year",
        "company_location"
    ]
]


In [ ]:
Clean job titles (normalize roles)

In [5]:
def normalize_job_title(title):
    title = title.lower()
    if "data analyst" in title:
        return "Data Analyst"
    elif "data scientist" in title:
        return "Data Scientist"
    elif "machine learning" in title or "ai" in title:
        return "AI / ML Engineer"
    elif "business analyst" in title:
        return "Business Analyst"
    else:
        return "Other"

df_canada["job_role"] = df_canada["job_title"].apply(normalize_job_title)


Convert salary USD → CAD

(Using 1 USD = 1.35 CAD — realistic and interview-safe)

In [6]:
USD_TO_CAD = 1.35
df_canada["salary_cad"] = df_canada["salary_in_usd"] * USD_TO_CAD


Map experience levels

In [7]:
experience_map = {
    "EN": "Entry-level",
    "MI": "Mid-level",
    "SE": "Senior-level",
    "EX": "Executive"
}

df_canada["experience_level"] = df_canada["experience_level"].map(experience_map)


Remove nulls & duplicates

In [9]:
df_canada.dropna(inplace=True)
df_canada.drop_duplicates(inplace=True)


Final check

In [10]:
df_canada.info()
df_canada.head()


<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 29 to 601
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         28 non-null     object 
 1   experience_level  28 non-null     object 
 2   employment_type   28 non-null     object 
 3   salary_in_usd     28 non-null     int64  
 4   work_year         28 non-null     int64  
 5   company_location  28 non-null     object 
 6   job_role          28 non-null     object 
 7   salary_cad        28 non-null     float64
dtypes: float64(1), int64(2), object(5)
memory usage: 2.0+ KB


,job_title,experience_level,employment_type,salary_in_usd,work_year,company_location,job_role,salary_cad
29,Machine Learning Manager,Senior-level,FT,117104,2020,CA,AI / ML Engineer,158090.40
82,Applied Data Scientist,Mid-level,FT,54238,2021,CA,Data Scientist,73221.30
106,Research Scientist,Mid-level,FT,187442,2021,CA,Other,253046.70
152,Data Scientist,Mid-level,FT,75774,2021,CA,Data Scientist,102294.90
155,Data Science Engineer,Senior-level,FT,127221,2021,CA,Other,171748.35


Export clean dataset for Tableau

In [11]:
df_canada.to_csv("cleaned_canada_jobs.csv", index=False)
